#Pre-processing files
#NetID Purging

In [1]:
from collections import defaultdict

In [2]:
fromNetID = 0.74 # 47 -> 74
rejectMin = 0.9 * fromNetID
rejectLimit = rejectMin + 0.01

def select_line(x):
    return ((x >= rejectMin) and (x < rejectLimit))

In [1]:
with open('edges_new.txt', 'w') as outfile:
    for line in open('edges.txt'):
        line_split = line.split()
        val = float(line_split[0])
        if not select_line(val):
            outfile.write(line_split[1]+" "+line_split[2]+"\n")
old_lines = open('edges.txt').read().count('\n')
new_lines = open('edges_new.txt').read().count('\n')
print "old lines", old_lines
print "new lines", new_lines
print (new_lines/float(old_lines))

NameError: name 'select_line' is not defined

##Normal Map Reduce - Node, Edges - formatting

In [15]:
def pre_proc_simple(output_file="nodes_simple.txt" ,input_file="edges_new.txt"):
    out_nodes_dict = defaultdict(list)
    for line in open(input_file):
        node, node_out = line.split()
        node = int(node)
        node_out = int(node_out)
        out_nodes_dict[node].append(node_out)
        if node_out not in out_nodes_dict:
            out_nodes_dict[node_out] = []
    #calculate intial node page rank    
    num_nodes = len(out_nodes_dict)
    init_page_rank = (1/float(num_nodes))
    with open(output_file, 'w') as outfile:
        for node, out_nodes in out_nodes_dict.iteritems():
            out_nodes_str = " ".join(str(x) for x in out_nodes)
            bound_conds_str = ""
            outfile.write(str(node)+"\t"+str(node)+","+str(init_page_rank)+","+out_nodes_str+","+bound_conds_str+"\n")

##Bloacked Map Reduce - Node, Edges, Boundary Conditions - formatting

In [3]:
#load blocks
cur_sum = 0
block_sums = []
for line in open('blocks.txt'):
    cur_sum += int(line)
    block_sums.append(cur_sum)
block_sums.append(block_sums[-1]*2)

def BlockIdOfNode(node_id):
    index = node_id/10000
    if(block_sums[index] < node_id):
        return index+1
    elif(index == 0 or block_sums[index-1] < node_id):
        return index
    else:
        return index-1

In [14]:
def pre_proc_avd(output_file="nodes.txt" ,input_file="edges_new.txt"):
    all_nodes = set()
    out_nodes_dict = defaultdict(list)
    bound_conds_dict = defaultdict(list)
    for line in open(input_file):
        node, node_out = line.split()
        node = int(node)
        node_out = int(node_out)
        out_nodes_dict[node].append(node_out)
        if (BlockIdOfNode(node) != BlockIdOfNode(node_out)):
            bound_conds_dict[node_out].append(node)
        all_nodes.add(node)
        all_nodes.add(node_out)
    #calc init page rank
    num_nodes = len(all_nodes)
    init_page_rank = (1/float(num_nodes))
    with open(output_file, 'w') as outfile:
        for node in all_nodes:
            out_nodes = out_nodes_dict[node]
            bound_conds = bound_conds_dict[node]
            out_nodes_str = " ".join(str(x) for x in out_nodes)
            bound_conds_str = " ".join(str(x) for x in bound_conds)            
            outfile.write(str(node)+"\t"+str(node)+","+str(init_page_rank)+","+out_nodes_str+","+bound_conds_str+"\n")

In [12]:
pre_proc_avd()

#Process test file - simple

In [16]:
pre_proc_simple(output_file="nodes_simple.txt" ,input_file="edges_new.txt")

In [18]:
pre_proc_simple(output_file="nodes_simple_test1.txt" ,input_file="edges_test1.txt")